In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.finance as finance
import datetime
import pandas as pd
import statsmodels.api as sm

C:\Users\mphan21\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [102]:
# Define data path 
ddir = 'C:/Users/mphan21/Google Drive/Academics/Spring 2018/BehavioralFinance/hmwk2/'

# Monthly equal weight
me_mon_ew = pd.read_csv(ddir+'me_mon_ew.csv')
me_mon_ew['date']=pd.to_datetime(me_mon_ew['date'], format="%Y%m").dt.strftime('%Y%m')
me_mon_ew.iloc[:,1:me_mon_ew.shape[1]]=me_mon_ew.iloc[:,1:me_mon_ew.shape[1]]/10

# Monthly value weight
me_mon_vw = pd.read_csv(ddir+'me_mon_vw.csv')
me_mon_vw['date']=pd.to_datetime(me_mon_vw['date'], format="%Y%m").dt.strftime('%Y%m')
me_mon_vw.iloc[:,1:me_mon_vw.shape[1]]=me_mon_vw.iloc[:,1:me_mon_vw.shape[1]]/10

# Jan 1963 - Dec 2017
aqr_ff_data = pd.read_csv(ddir + 'aqr_ff_data.csv')
aqr_ff_data['date'] = pd.to_datetime(aqr_ff_data['date'], format="%Y%m").dt.strftime('%Y%m')

# Fama French Monthly
fama_french = pd.read_csv(ddir+'fama_french.csv')
fama_french.columns = ['date', 'mktrf', 'smb', 'hml', 'rf', 'umd']
fama_french['date'] = pd.to_datetime(fama_french['date'], format='%Y%m%d').dt.strftime('%Y%m')

# Problem 1

## a) average monthly returns and z-statistics for each portfolio.

In [51]:
me_mon_ew.head()

,date,Lo 10,2-Dec,3-Dec,4-Dec,5-Dec,6-Dec,7-Dec,8-Dec,9-Dec,Hi 10
0,192607,-0.145,0.029,-0.015,0.133,0.124,0.198,0.155,0.138,0.338,0.329
1,192608,0.512,0.259,0.403,0.315,0.272,0.472,0.160,0.163,0.098,0.370
2,192609,0.093,-0.187,-0.227,-0.053,0.007,-0.007,-0.164,0.064,-0.086,0.067
3,192610,-0.484,-0.177,-0.336,-0.483,-0.298,-0.280,-0.345,-0.327,-0.347,-0.243
4,192611,-0.078,-0.032,-0.029,0.465,0.324,0.357,0.382,0.295,0.361,0.270


In [52]:
# Returns
avg_mon_ew=me_mon_ew[me_mon_ew.columns[1:me_mon_ew.shape[1]]].mean()
avg_mon_ew

Lo 10    0.180088
2-Dec    0.136317
3-Dec    0.129316
4-Dec    0.122953
5-Dec    0.116696
6-Dec    0.119068
7-Dec    0.110369
8-Dec    0.107329
9-Dec    0.101801
Hi 10    0.089945
dtype: float64

In [53]:
# Std
std_mon_ew=me_mon_ew[me_mon_ew.columns[1:me_mon_ew.shape[1]]].std()
std_mon_ew
z_mon_ew=avg_mon_ew/std_mon_ew
z_mon_ew

Lo 10    0.168876
2-Dec    0.149104
3-Dec    0.157449
4-Dec    0.158854
5-Dec    0.161468
6-Dec    0.171440
7-Dec    0.167129
8-Dec    0.171028
9-Dec    0.170611
Hi 10    0.166532
dtype: float64

## b) CAPM alphas and t-statistics

In [54]:
mon_ew_combine=pd.merge(me_mon_ew,fama_french,on='date')
mon_ew_combine.head()

,date,Lo 10,2-Dec,3-Dec,4-Dec,5-Dec,6-Dec,7-Dec,8-Dec,9-Dec,Hi 10,mktrf,smb,hml,rf,umd
0,192607,-0.145,0.029,-0.015,0.133,0.124,0.198,0.155,0.138,0.338,0.329,0.0296,-0.0230,-0.0287,0.0022,NaN
1,192608,0.512,0.259,0.403,0.315,0.272,0.472,0.160,0.163,0.098,0.370,0.0264,-0.0140,0.0419,0.0025,NaN
2,192609,0.093,-0.187,-0.227,-0.053,0.007,-0.007,-0.164,0.064,-0.086,0.067,0.0036,-0.0132,0.0001,0.0023,NaN
3,192610,-0.484,-0.177,-0.336,-0.483,-0.298,-0.280,-0.345,-0.327,-0.347,-0.243,-0.0324,0.0004,0.0051,0.0032,NaN
4,192611,-0.078,-0.032,-0.029,0.465,0.324,0.357,0.382,0.295,0.361,0.270,0.0253,-0.0020,-0.0035,0.0031,NaN


In [96]:
alphas = [0] * 10
mon_ew_resid = me_mon_ew
for portfolio in range(1,11):
    print("Portfolio: "+mon_ew_combine.columns[portfolio])
    cur_mod=sm.OLS(mon_ew_combine[mon_ew_combine.columns[portfolio]]-mon_ew_combine['rf'], 
                   sm.add_constant(mon_ew_combine['mktrf']), missing='drop').fit()
    alphas[portfolio-1] = cur_mod.params[0]
    mon_ew_resid[mon_ew_combine.columns[portfolio]]=cur_mod.resid
    print(cur_mod.summary())
    print(portfolio)

Portfolio: Lo 10
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                     1148.
Date:                Mon, 19 Feb 2018   Prob (F-statistic):          9.33e-173
Time:                        23:50:21   Log-Likelihood:                -1234.8
No. Observations:                1098   AIC:                             2474.
Df Residuals:                    1096   BIC:                             2484.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0826      0.023   

In [97]:
sigma_resid=mon_ew_resid.iloc[:,1:mon_ew_resid.shape[1]].cov()
sigma_resid

,Lo 10,2-Dec,3-Dec,4-Dec,5-Dec,6-Dec,7-Dec,8-Dec,9-Dec,Hi 10
Lo 10,0.555582,0.326793,0.227802,0.190871,0.133383,0.099689,0.082940,0.036931,0.017338,-0.014050
2-Dec,0.326793,0.249085,0.174076,0.150726,0.108760,0.082173,0.066739,0.033915,0.014537,-0.009497
3-Dec,0.227802,0.174076,0.144868,0.118576,0.088386,0.068470,0.054340,0.031933,0.014609,-0.007458
4-Dec,0.190871,0.150726,0.118576,0.115219,0.080624,0.063981,0.051339,0.030721,0.014533,-0.005579
5-Dec,0.133383,0.108760,0.088386,0.080624,0.071721,0.051384,0.041950,0.027465,0.014373,-0.003227
6-Dec,0.099689,0.082173,0.068470,0.063981,0.051384,0.050121,0.034838,0.024598,0.014686,-0.001463
7-Dec,0.082940,0.066739,0.054340,0.051339,0.041950,0.034838,0.037862,0.021780,0.013188,0.000148
8-Dec,0.036931,0.033915,0.031933,0.030721,0.027465,0.024598,0.021780,0.023974,0.012835,0.001789
9-Dec,0.017338,0.014537,0.014609,0.014533,0.014373,0.014686,0.013188,0.012835,0.017469,0.004161
Hi 10,-0.014050,-0.009497,-0.007458,-0.005579,-0.003227,-0.001463,0.000148,0.001789,0.004161,0.009627


In [98]:
#alpha*sigma^-1*alpha
resid_quad = np.dot(alphas,pd.DataFrame(np.linalg.inv(sigma_resid.values), 
                           sigma_resid.columns, sigma_resid.index).dot(alphas))

In [99]:
#(1+E(F)Cov(F)^-1E(F))
mu_factors = mon_ew_combine['mktrf'].mean()
sigma_factors = mon_ew_combine['mktrf'].var()
factors_quad=(1+mu_factors*1/sigma_factors*mu_factors)

In [100]:
N=10
T=mon_ew_combine.shape[0]
K=1

F_stat = (T-N-K)/N*factors_quad*resid_quad

In [101]:
# Reject that the sized ranked deciles are priced by the market portfolio
from scipy.stats import f
pvalue=1-f.cdf(F_stat, N, T-N-K)
pvalue

1.1102230246251565e-16

# Problem 2 - Value Weighted 

In [86]:
mon_vw_combine=pd.merge(me_mon_vw,fama_french,on='date')
mon_vw_combine.head()

,date,Lo 10,2-Dec,3-Dec,4-Dec,5-Dec,6-Dec,7-Dec,8-Dec,9-Dec,Hi 10,mktrf,smb,hml,rf,umd
0,192607,-0.012,0.052,-0.005,0.131,0.121,0.204,0.158,0.129,0.353,0.371,0.0296,-0.0230,-0.0287,0.0022,NaN
1,192608,0.133,0.255,0.400,0.320,0.281,0.445,0.161,0.149,0.061,0.379,0.0264,-0.0140,0.0419,0.0025,NaN
2,192609,0.059,-0.200,-0.201,-0.046,-0.006,0.014,-0.202,0.074,-0.077,0.125,0.0036,-0.0132,0.0001,0.0023,NaN
3,192610,-0.433,-0.201,-0.325,-0.435,-0.293,-0.248,-0.360,-0.326,-0.336,-0.256,-0.0324,0.0004,0.0051,0.0032,NaN
4,192611,-0.330,-0.023,0.008,0.474,0.364,0.344,0.363,0.305,0.386,0.240,0.0253,-0.0020,-0.0035,0.0031,NaN


In [103]:
alphas = [0] * 10
mon_vw_resid = me_mon_vw
for portfolio in range(1,11):
    print("Portfolio: "+mon_vw_combine.columns[portfolio])
    cur_mod=sm.OLS(mon_vw_combine[mon_vw_combine.columns[portfolio]]-mon_vw_combine['rf'], 
                   sm.add_constant(mon_vw_combine['mktrf']), missing='drop').fit()
    alphas[portfolio-1] = cur_mod.params[0]
    mon_vw_resid[mon_vw_combine.columns[portfolio]]=cur_mod.resid
    print(cur_mod.summary())
    print(portfolio)

Portfolio: Lo 10
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.582
Method:                 Least Squares   F-statistic:                     1526.
Date:                Mon, 19 Feb 2018   Prob (F-statistic):          7.96e-210
Time:                        23:50:55   Log-Likelihood:                -1069.9
No. Observations:                1098   AIC:                             2144.
Df Residuals:                    1096   BIC:                             2154.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0432      0.020   

In [104]:
alphas

[0.0432118175341673,
 0.031944054783501583,
 0.034773781168979639,
 0.035488489513523326,
 0.032321239143089868,
 0.035906335409233638,
 0.031787983831820951,
 0.031393320365700535,
 0.027948466709804703,
 0.024526863117559059]

In [94]:
sigma_resid=mon_vw_resid.iloc[:,1:mon_vw_resid.shape[1]].cov()
sigma_resid

resid_quad = np.dot(alphas,pd.DataFrame(np.linalg.inv(sigma_resid.values), 
                           sigma_resid.columns, sigma_resid.index).dot(alphas))

#(1+E(F)Cov(F)^-1E(F))
mu_factors = mon_vw_combine['mktrf'].mean()
sigma_factors = mon_vw_combine['mktrf'].var()
factors_quad=(1+mu_factors*1/sigma_factors*mu_factors)

N=10
T=mon_vw_combine.shape[0]
K=1

F_stat = (T-N-K)/N*factors_quad*resid_quad

pvalue=1-f.cdf(F_stat, N, T-N-K)
pvalue

1.1102230246251565e-16

# Problem 3- Daily Data